### Transform key values from the extracted Certificate Holders Statement pdf files into csv file.

In [1]:
import fitz  # PyMuPDF
import re
import os
import csv

# Directory where PDF files are stored
pdf_directory = r"C:\Users\Rex Fuentes\Documents\Data Engineering Case Study\citigroup_case_study\certfc8_holdrs_st8mnt"
csv_filename = r"C:\Users\Rex Fuentes\Documents\Data Engineering Case Study\citigroup_case_study\breakdown_of_principal.csv"

# Check if the directory exists, and if not, create it
csv_directory = os.path.dirname(csv_filename)
if not os.path.exists(csv_directory):
    os.makedirs(csv_directory)

# Regular expression patterns for extracting data
distribution_date_pattern = r"Distribution Date:\s*(\d{1,2}/\d{1,2}/\d{4})"
determination_date_pattern = r"(\d{1,2}/\d{1,2}/\d{4})\s*Determination Date:"
total_principal_pattern = r"Total Principal Funds Available:\s*\$?([\d,]+\.\d{2})"
scheduled_principal_pattern = r"Scheduled Principal\s*\$?([\d,]+\.\d{2})"
curtailments_pattern = r"Curtailments\s*\$?\(?([\d,]+\.\d{2})\)?"
prepayments_pattern = r"Prepayments in Full\s*\$?([\d,]+\.\d{2})"
net_liquidation_proceeds_pattern = r"Net Liquidation Proceeds\s*\$?\(?([\d,]+\.\d{2})\)?"

def extract_value_from_text(pattern, text):
    match = re.search(pattern, text)
    if match:
        value = match.group(1).replace(',', '')
        if '(' in match.group(0) and ')' in match.group(0):
            value = '-' + value
        return value
    elif 'Net Liquidation Proceeds' in text or 'Curtailments' in text or 'Prepayments in Full' in text:
        return '0.00'
    else:
        return "Not found"

def extract_data_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)

    # Check for the unique marker "Page 1" on the first page
    page_one_text = pdf_document[0].get_text() if len(pdf_document) > 0 else ""
    page_one_present = re.search(r"Page 1\b", page_one_text) is not None

    # Use page 6 if page 1 is present, else use page 5
    target_page_index = 5 if page_one_present else 4

    # Extract text from the target page
    if len(pdf_document) > target_page_index:
        page_text = pdf_document[target_page_index].get_text()
        distribution_date = extract_value_from_text(distribution_date_pattern, page_text)
        determination_date = extract_value_from_text(determination_date_pattern, page_text)
        total_principal = extract_value_from_text(total_principal_pattern, page_text)
        scheduled_principal = extract_value_from_text(scheduled_principal_pattern, page_text)
        curtailments = extract_value_from_text(curtailments_pattern, page_text)
        prepayments = extract_value_from_text(prepayments_pattern, page_text)
        net_liquidation_proceeds = extract_value_from_text(net_liquidation_proceeds_pattern, page_text)
    else:
        # Set all values to "Not found" if the target page does not exist
        distribution_date = determination_date = total_principal = scheduled_principal = curtailments = prepayments = net_liquidation_proceeds = "Not found"

    pdf_document.close()
    return (
        distribution_date, determination_date, total_principal, scheduled_principal, curtailments, prepayments, net_liquidation_proceeds
    )

# Write the headers to the CSV file
with open(csv_filename, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    headers = [
        'Filename', 'Distribution Date', 'Determination Date', 'Total Principal Funds Available',
        'Scheduled Principal', 'Curtailments', 'Prepayments in Full', 'Net Liquidation Proceeds'
    ]
    csv_writer.writerow(headers)

    # Iterate over each PDF file in the directory
    for filename in os.listdir(pdf_directory):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, filename)
            data = extract_data_from_pdf(pdf_path)
            csv_writer.writerow([filename] + list(data))

print("Transformation complete. Data saved to", csv_filename)


Extraction complete. Data saved to C:\Users\Rex Fuentes\Documents\breakdown_of_principal.csv


### Loads the csv file to the citigroup database

In [4]:
import pandas as pd
from sqlalchemy import create_engine
import os
import configparser

# Function to drop entirely empty columns
def drop_empty_columns(df):
    return df.dropna(axis=1, how='all')

# Read database credentials and file directory from a config file
config = configparser.ConfigParser()
config.read('config.ini')

# Database connection URL
db_connection_url = f"postgresql+psycopg2://{config['database']['user']}:{config['database']['password']}@{config['database']['host']}:{config['database']['port']}/{config['database']['database']}"

# Create a SQLAlchemy engine
engine = create_engine(db_connection_url)

# Directory where your CSV files are stored
csv_folder_path = config['files']['csv_file_path_of_transformed_pdf']

# Check if the path is a directory
if os.path.isdir(csv_folder_path):
    # Iterate over each file in the folder and insert data into the database
    for csv_file in os.listdir(csv_folder_path):
        if csv_file.endswith('.csv'):
            file_path = os.path.join(csv_folder_path, csv_file)

            # Use Pandas to load the CSV file
            df = pd.read_csv(file_path)

            # Drop columns that are entirely empty
            df = drop_empty_columns(df)

            # Use 'to_sql' to insert the data into the database, it creates a table if it does not exist
            df.to_sql('transformed_pdf', engine, if_exists='append', index=False)

    print("Loading data completed. All empty columns were dropped.")
else:
    print(f"The provided path is not a directory: {csv_folder_path}")


Loading data completed. All empty columns were dropped.
